**Load** the required packages

In [1]:
!pip install --upgrade numexpr --quiet

!pip install pandas --quiet
!pip install datetime --quiet
!pip install feedparser --quiet
!pip install textblob --quiet
!pip install yfinance --quiet
!pip install requests --quiet

**Define a function to fetch sentinment analysis for the Nadex Stocks**
1. Start with loading the Yahoo Finance Feed
2. Determine the Sentiment
3. Collapse per day and average the Sentiment

In [2]:
import pandas as pd
import feedparser
from textblob import TextBlob
from datetime import timedelta

from typing import List, Tuple

def fetch_sentiment_records() -> List:
    """
    Fetch all RSS entries and compute a sentiment score per item.
    """
    records = []
    for ticker, url in TICKERS_AND_FEEDS.items():
        feed = feedparser.parse(url)
        for entry in feed.entries:
            pub_ts = pd.to_datetime(entry.get("published", ""), utc=True)
            score = TextBlob(f"{entry.title}. {entry.get('summary','')}").sentiment.polarity
            records.append({
                "ticker": ticker,
                "Date": pub_ts.date(),
                "score": score
            })
    return records

def get_recent_sentiment(days=180) -> pd.DataFrame:
    """Return mean daily sentiment per ticker over the last `days` days."""
    df = (
        pd.DataFrame(fetch_sentiment_records())
          .groupby(["ticker", "Date"], as_index=False)
          .score.mean()
    )
    cutoff = (pd.Timestamp.today().normalize() - timedelta(days=days)).date()
    return df[df.Date.ge(cutoff)].reset_index(drop=True)

**Define a function to get the pricing data**

In [3]:
import yfinance as yf
from typing import List, Tuple

def fetch_price(ticker) -> pd.DataFrame:
    df = yf.download(ticker, period="6mo", interval="1d", auto_adjust=True, progress=False)
    df.columns = df.columns.get_level_values(0)
    df.reset_index(inplace=True) 
    df.columns.name = None
    df['ticker'] = ticker
    return df

def merge_sentiment_scores(ticker_price_data, sentiment_df):
    """
    ticker_price_data: list of (ticker, price_df)
    sentiment_df: DataFrame with ['ticker','Date','score']
    
    Returns: dict where key=ticker, value=merged DataFrame for that ticker
    """
    # 1) Normalize sentiment Date dtype once
    sentiment = sentiment_df.copy()
    sentiment['Date'] = pd.to_datetime(sentiment['Date'])
    
    result = {}
    for ticker, price_df in ticker_price_data:
        # 2) Ensure price_df.Date is a datetime64 column
        df = price_df.reset_index() if price_df.index.name == 'Date' else price_df.copy()
        if df['Date'].dtype == object:
            df['Date'] = pd.to_datetime(df['Date'])
        
        # 3) Filter sentiment to this ticker
        daily = sentiment[sentiment['ticker'] == ticker][['Date', 'score']]
        
        # 4) Merge and store
        merged_df = df.merge(daily, on='Date', how='left')
        merged_df['ticker'] = ticker
        
        result[ticker] = merged_df
    
    return result


**Compute the Technical Indicators**

1. Compute the MACD
2. Compute the RSI
3. Compute the ATR

In [4]:
import pandas as pd

def compute_macd(df: pd.DataFrame) -> pd.DataFrame:
    """
    Return a new DataFrame with EMA12, EMA26, MACD, Signal, and MACD_hist added.
    """
    df = df.copy()
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD']  = df['EMA12'] - df['EMA26']
    df['Signal']   = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['Signal']
    return df

def compute_rsi(df: pd.DataFrame, window: int = 14) -> pd.DataFrame:
    """
    Return a new DataFrame with a 14-period RSI column added.
    """
    df = df.copy()
    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/window, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/window, adjust=False).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

def compute_atr(df: pd.DataFrame, window: int = 14) -> pd.DataFrame:
    """
    Return a new DataFrame with a 14-period ATR column added.
    """
    df = df.copy()
    prev_close = df['Close'].shift(1)
    tr1 = df['High'] - df['Low']
    tr2 = (df['High'] - prev_close).abs()
    tr3 = (df['Low']  - prev_close).abs()
    df['ATR'] = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1).rolling(window=window).mean()
    return df


**Load the strike prices**

In [5]:
import pandas as pd

def load_strikes(
    path: str,
    strike_col: str = "strike",
    decimals: int = 4
) -> pd.DataFrame:
    """
    1. Reads a CSV from `path`
    2. Normalizes column names to lowercase & stripped
    3. Ensures there's a numeric `strike_col`, stripping whitespace, converting to float, and rounding
    
    Returns the cleaned DataFrame.
    """
    return (
        pd.read_csv(path)
          # normalize headers: strip whitespace, lowercase
          .rename(columns=lambda c: c.strip().lower())
          # strip & convert the strike column to numeric, then round
          .assign(**{
              strike_col: lambda df: (
                  pd.to_numeric(
                      df[strike_col]
                        .astype(str)
                        .str.strip(),
                      errors="raise"
                  )
                  .round(decimals)
              )
          })
    )


**Determine the Trading Signal**

1. Determine the Trend
2. Check the Momentum
3. Determine the Trigger
4. Check the Volatility
5. Check the Sentiment
6. Generate the signal

In [6]:
import pandas as pd

def determine_trend(row):
    """
    Returns (trend_str, ema_diff) where ema_diff = EMA12 − EMA26.
    """
    diff = row.EMA12 - row.EMA26
    if (row.EMA12 > row.EMA26) and (row.Close > row.EMA12) and (row.Close > row.EMA26):
        trend = 'up'
    elif (row.EMA12 < row.EMA26) and (row.Close < row.EMA12) and (row.Close < row.EMA26):
        trend = 'down'
    else:
        trend = 'sideways'
    return trend, diff

def momentum_check(row, trend):
    """
    Returns (ok, rsi) where ok is True/False and rsi is the raw RSI value.
    """
    rsi = row.RSI
    if trend == 'up':
        ok = (rsi > 50) and ((row.MACD > row.Signal) or (row.MACD_hist > 0))
    elif trend == 'down':
        ok = (rsi < 50) and ((row.MACD < row.Signal) or (row.MACD_hist < 0))
    else:
        ok = False
    return ok, rsi

def signal_trigger(row, trend):
    """
    Returns (ok, macd_diff) where macd_diff = MACD − Signal.
    """
    macd_diff = row.MACD - row.Signal
    if trend == 'up':
        ok = (macd_diff > 0) and (row.RSI > 50)
    elif trend == 'down':
        ok = (macd_diff < 0) and (row.RSI < 50)
    else:
        ok = False
    return ok, macd_diff

def volatility_check(row, strike_diff):
    """
    Returns (ok, strike_diff) so you have the raw distance too.
    """
    ok = strike_diff <= 0.5 * row.ATR
    return ok, strike_diff

def sentiment_check(row, threshold=0.2):
    """
    Returns (ok, score) so you keep the raw sentiment polarity.
    """
    score = row.score
    ok = score > -threshold
    return ok, score

def signal_detail_for_row(row, per_ticker, expiry="EOD"):
    df = per_ticker[row.ticker]
    last = df.iloc[-1]

    trend, trend_val = determine_trend(last)
    momentum_ok, momentum_val = momentum_check(last, trend)
    sig_ok, signal_val = signal_trigger(last, trend)
    strike_diff = abs(row.strike - last.Close)
    vol_ok, vol_val = volatility_check(last, strike_diff)
    sent_ok, sent_val = sentiment_check(last)

    # build recommendation + contract price
    if (trend == 'sideways'
        or not momentum_ok
        or not sig_ok
        or not vol_ok
        or not sent_ok
    ):
        rec   = "No trade"
        price = pd.NA
    else:
        direction = "Buy" if trend == "up" else "Sell"
        price     = 10 * (0.5 - (strike_diff / (2 * last.ATR)))
        rec       = direction

    return pd.Series({
        "Date":           pd.Timestamp.now().strftime("%d-%b-%y"),
        "Ticker":         row.ticker,
        "Strike":         row.strike,
        "EMA12":          last.EMA12,
        "EMA26":          last.EMA26,
        "MACD":           last.MACD,
        "RSI":            last.RSI,
        "ATR":            last.ATR,
        "Recommendation": rec,
        "ContractPrice":  price,
        "Trend":          trend_val,
        "Momentum":       momentum_val,
        "Signal":         signal_val,
        "Volatility":     vol_val,
        "Sentiment":      sent_val,
    })

def generate_detailed_signals(per_ticker: dict[str,pd.DataFrame],
                              strikes_df: pd.DataFrame) -> pd.DataFrame:
    """
    Applies signal_detail_for_row to every strike, returning a DataFrame
    with separate columns for both the boolean/string and the raw measure.
    """
    return strikes_df.apply(
        lambda r: signal_detail_for_row(r, per_ticker),
        axis=1
    )    

def generate_trade_signal(df: pd.DataFrame, asset_symbol: str, strike_price: float, expiry="EOD") -> str:
    """
    Look only at the *last* row of df and decide on a trade recommendation.
    """
    last = df.iloc[-1]
    trend = determine_trend(last)
    if trend == 'sideways':
        return "No trade: Trend unclear."
    if trend == 'down':
        # optional debug print
        print(f"Down trend for {asset_symbol} at {strike_price:.4f}")

    if not momentum_check(last, trend):
        return "No trade: Momentum not aligned."
    if not signal_trigger(last, trend):
        return "No trade: No signal trigger."

    strike_diff = abs(strike_price - last.Close)
    if not volatility_check(last, strike_diff):
        return "No trade: Strike too far based on ATR."
    if not sentiment_check(last):
        return "No trade: Negative sentiment contradicts signal."

    # crude price proxy
    price_est = 10 * (0.5 - (strike_diff / (2 * last.ATR)))
    direction = "Buy" if trend == 'up' else "Sell"
    return (
        f"{direction} {asset_symbol} @ {strike_price:.4f} (EOD) ≈ ${price_est:.1f}. "
        f"{trend.title()} trend, momentum/signal OK, volatility OK, sentiment OK."
    )

def generate_all_signals(
    per_ticker: dict[str, pd.DataFrame], 
    strikes_df: pd.DataFrame
) -> pd.DataFrame:
    """
    For each row in strikes_df, look up its ticker-DF in per_ticker and
    call generate_trade_signal.  Returns a DataFrame of recommendations.
    """
    out = (
        strikes_df
        .assign(
            Date=pd.Timestamp.now().strftime('%d-%b-%y'),
            Signal=lambda d: d.apply(
                lambda r: generate_trade_signal(
                    per_ticker[r.ticker],
                    r.ticker,
                    r.strike
                ),
                axis=1
            )
        )
        .rename(columns={'ticker':'Ticker', 'strike':'Strike'})
        [['Date','Ticker','Strike','Signal']]
    )
    return out

**Define a function to upload the recommendations to S3**

In [7]:
import io
import boto3
import pandas as pd
from botocore.exceptions import ClientError
from botocore import UNSIGNED
from botocore.config import Config

from typing import Iterable, List, Dict

def create_s3_clients(
    profile: str = "default", region: str = "us-east-1"
) -> Dict[str, boto3.client]:
    session = boto3.Session(profile_name=profile, region_name=region)
    return {
        "public": session.client(
            "s3",
            config=Config(signature_version=UNSIGNED),
            region_name=region,
        ),
        "private": session.client("s3"),
        "resource": session.resource("s3"),
    }

def get_bucket(resource: boto3.resource, name: str):
    return resource.Bucket(name)

def upload_df_to_s3(
    df: pd.DataFrame,
    bucket: str,
    key: str,
    region: str = None) -> None:
    """
    Uploads a DataFrame to S3 as CSV. Verifies bucket existence first.
    
    Parameters
    ----------
    df : pd.DataFrame
    bucket : str
        Name of the S3 bucket (no leading/trailing spaces).
    key : str
        S3 object key, e.g. "csv/2025-06-28-results.csv"
    region : str, optional
        AWS region where the bucket resides.
    """
    bucket = bucket.strip()
    s3 = boto3.client('s3', region_name=region)

    try:
        s3.head_bucket(Bucket=bucket)
    except ClientError as e:
        code = e.response['Error']['Code']
        msg = e.response['Error']['Message']
        raise RuntimeError(
            f"Could not access bucket '{bucket}' (region={region}): {msg} (code {code})"
        ) from e

    buffer = io.StringIO()
    df.to_csv(buffer, index=False)
    buffer.seek(0)

    try:
        s3.put_object(Bucket=bucket, Key=key, Body=buffer.getvalue())
        print(f"✅ Uploaded to s3://{bucket}/{key}")
    except ClientError as e:
        raise RuntimeError(
            f"Failed to upload CSV to s3://{bucket}/{key}: "
            f"{e.response['Error']['Message']}"
        ) from e

**Define a function to run the Pipeline**
1. Get the news and create the Sentiment Analysis
2. Collect pricing data

In [24]:
import pandas as pd
from datetime import date, datetime

from typing import List

def run_prediction_pipeline(tickers_and_feeds: List,
                           bucket_name: str) -> pd.DataFrame:
    """
    Fetch price & sentiment, compute indicators, load strikes,
    and return a DataFrame of trade signals.
    """
    clients = create_s3_clients()
    public_s3 = clients["public"]
    private_s3 = clients["private"]
    s3_resource = clients["resource"]
    buckets = {
        "daily":  get_bucket(s3_resource, bucket_name),
    }

    ticker_price_data = [
        (ticker, fetch_price(ticker))
        for ticker in tickers_and_feeds
    ]

    merged = merge_sentiment_scores(ticker_price_data,
                                    get_recent_sentiment(DAYS_TO_INCLUDE))
    processed = {
        ticker: (
            df
              .pipe(compute_macd)
              .pipe(compute_rsi)
              .pipe(compute_atr)
        )
        for ticker, df in merged.items()
    }
    strikes_df = load_strikes("contracts.csv")
    signals_df = generate_detailed_signals(processed, strikes_df)

    today_str = date.today().strftime('%Y%m%d')

    s3_key = f"recommendations/{today_str}.csv"
    upload_df_to_s3(
        signals_df,    
        bucket_name,   
        s3_key         
    )
    
    return signals_df

**Define a function to show interesting trades**

In [25]:
def show_interesting_trades(df: pd.DataFrame) -> None:
    interesting_trades_df = df[
        ~df['Recommendation']
            .str.contains("No trade", case=False, na=False)
    ]
    if not interesting_trades_df.empty:
        print(
            tabulate(
                interesting_trades_df[["Date","Ticker","Recommendation","Strike","ContractPrice"]],
                headers='keys',
                tablefmt='fancy_grid',
                showindex=False,        
                maxcolwidths=200  
            )
        )
    else:
        print("No trades recommended today")

**Run the pipeine**

In [26]:
from tabulate import tabulate

DAYS_TO_INCLUDE = 90 
TICKERS_AND_FEEDS = {
    'CL=F': 'https://finance.yahoo.com/rss/headline?s=CL=F',
    'ES=F': 'https://finance.yahoo.com/rss/headline?s=ES=F',
    'GC=F': 'https://finance.yahoo.com/rss/headline?s=GC=F',
    'NQ=F': 'https://finance.yahoo.com/rss/headline?s=NQ=F',
    'RTY=F': 'https://finance.yahoo.com/rss/headline?s=RTY=F',
    'YM=F': 'https://finance.yahoo.com/rss/headline?s=YM=F',
    'AUDUSD=X': 'https://finance.yahoo.com/rss/headline?s=AUDUSD=X',
    'EURJPY=X': 'https://finance.yahoo.com/rss/headline?s=EURJPY=X',
    'EURUSD=X': 'https://finance.yahoo.com/rss/headline?s=EURUSD=X',
    'GBPJPY=X': 'https://finance.yahoo.com/rss/headline?s=GBPJPY=X',
    'GBPUSD=X': 'https://finance.yahoo.com/rss/headline?s=GBPUSD=X',
    'USDCAD=X': 'https://finance.yahoo.com/rss/headline?s=USDCAD=X',
    'USDCHF=X': 'https://finance.yahoo.com/rss/headline?s=USDCHF=X',
    'USDJPY=X': 'https://finance.yahoo.com/rss/headline?s=USDJPY=X'
}

show_interesting_trades(
    run_prediction_pipeline(TICKERS_AND_FEEDS, 
                            "nadex-daily-results"))

✅ Uploaded to s3://nadex-daily-results/recommendations/20250701.csv
╒═══════════╤══════════╤══════════════════╤══════════╤═════════════════╕
│ Date      │ Ticker   │ Recommendation   │   Strike │   ContractPrice │
╞═══════════╪══════════╪══════════════════╪══════════╪═════════════════╡
│ 01-Jul-25 │ ES=F     │ Buy              │   6280.8 │         3.18106 │
├───────────┼──────────┼──────────────────┼──────────┼─────────────────┤
│ 01-Jul-25 │ ES=F     │ Buy              │   6268.8 │         4.00318 │
├───────────┼──────────┼──────────────────┼──────────┼─────────────────┤
│ 01-Jul-25 │ ES=F     │ Buy              │   6256.8 │         4.8253  │
├───────────┼──────────┼──────────────────┼──────────┼─────────────────┤
│ 01-Jul-25 │ ES=F     │ Buy              │   6244.8 │         4.35258 │
├───────────┼──────────┼──────────────────┼──────────┼─────────────────┤
│ 01-Jul-25 │ ES=F     │ Buy              │   6232.8 │         3.53046 │
├───────────┼──────────┼──────────────────┼──────────┼──